In [1]:
import numpy as np
import os
import tensorflow as tf
import sys

############ Tensforflow config ############
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.compat.v1.disable_eager_execution()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

############## Import modules ##############
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [12]:
from modules import ucr_loader, utils
# get all datasets
root = '../../../data/'
dataset_dict = ucr_loader.get_datasets(root, prefix='**/')
# retrieve data
dataset_name = 'FordA'
trainX, trainY, testX, testY = ucr_loader.load_data(dataset_dict[dataset_name])
# preprocess data
trainX, trainY, testX, testY = ucr_loader.preprocess_data(trainX, trainY, testX, testY, normalize=False, standardize=True)
# additional preprocessing
trainX, trainY, valX, valY = utils.perform_datasplit(trainX, trainY, test_split=0.3)
n_classes = len(np.unique(trainY))
# Shapes
print('TrainX:', trainX.shape)
print('ValX:', valX.shape)
print('TestX:', testX.shape)
print('Classes:', n_classes)

# uncomment to use the save folders
#model_dir, result_dir = utils.maybe_create_dirs(dataset_name, root='../../../', dirs=['models', 'results'], return_paths=True, verbose=1)

TrainX: (2520, 500, 1)
ValX: (1081, 500, 1)
TestX: (3601, 500, 1)
Classes: 2


Training using a model with privacy preserving method (Differential Privacy)
- hyper parameters to ad noise and clip gradients are used for differential privacy

In [13]:
epochs, batch_size, learning_rate = 100, 32, 0.2
#privacy parameter
l2_norm_clip, noise_multiplier, num_microbatches = 1.0, 0.2, batch_size
if batch_size % num_microbatches != 0:
    num_microbatches = batch_size

In [14]:
train_size, val_size = int(trainX.shape[0]/batch_size) * batch_size, int(valX.shape[0]/batch_size) * batch_size
trainXm, trainYm, valXm, valYm = trainX[:train_size], trainY[:train_size], valX[:val_size], valY[:val_size]

In [ ]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=train_size, batch_size=batch_size, noise_multiplier=noise_multiplier, epochs=epochs, delta=1/train_size)

In [ ]:
from modules.model_definition import AlexNet
from modules.DM import trainer_differential_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer

# uncomment to save the model
model_path = None# os.path.join(model_dir, 'AlexNet_differential_1d_batch-' + str(batch_size) + '_l2-' + str(l2_norm_clip) + '_noise-' + str(noise_multiplier) + '.h5')
model = AlexNet().build_1d(trainX.shape[1:], n_classes, activation='softmax', verbose=1)
optimizer = DPKerasSGDOptimizer(l2_norm_clip=l2_norm_clip, noise_multiplier=noise_multiplier, num_microbatches=num_microbatches, learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
trainer_differential_privacy.traintf(sess, model, trainXm, trainYm, validation_data=(valXm, valYm), 
                                        epochs=epochs, batch_size=batch_size, model_path=model_path, verbose=1)

In [ ]:
test_size = int(testX.shape[0]/batch_size) * batch_size
testXm = testX
if test_size < testX.shape[0]:
    fill = np.zeros((batch_size - (testX.shape[0] - test_size), *testX.shape[1:]))
    testXm = np.concatenate([testX, fill], axis=0)
# uncomment to save the model
report_path = None#os.path.join(result_dir, 'AlexNet_differential_1d_batch-' + str(batch_size) + '_l2-' + str(l2_norm_clip) + '_noise-' + str(noise_multiplier) + '_report.txt')
preds = np.argmax(model.predict(testXm, batch_size=batch_size, verbose=1), axis=1)
utils.compute_classification_report(testY, preds[:testX.shape[0]], save=report_path, verbose=1)